In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from tqdm import trange
from src.svgd import SVGD
from src.gsvgd import FullGSVGDBatch
from src.kernel import RBF, BatchRBF
from src.manifold import Grassmann
import torch.autograd as autograd
from src.s_svgd import SlicedSVGD
from src.diffusion import Diffusion
from src.rand_mysvgd import mySVGD
import pickle
import argparse
import time
import numpyro
from numpyro.infer import MCMC, NUTS
import numpyro.distributions as npr_dist
import jax.random as random
import jax.numpy as jnp
import jax
from src.Tmy_svgd import etmySVGD

/home/zhoujk/anaconda3/envs/AG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description="Running xshaped experiment.")
parser.add_argument("--dim", type=int, default=50, help="dimension")
parser.add_argument("--effdim", type=int, default=5, help="dimension")
parser.add_argument("--lr", type=float, default=0.0001, help="learning rate")
parser.add_argument("--lr_g", type=float, default=0.0001, help="learning rate for S-SVGD")
parser.add_argument(
    "--delta", type=float, default=0.1, help="stepsize for projections"
)
parser.add_argument(
    "--T", type=float, default=1e-4, help="noise multiplier for projections"
)
parser.add_argument("--nparticles", type=int, default=2500, help="no. of particles")
parser.add_argument("--epochs", type=int, default=20000, help="no. of epochs")
parser.add_argument("--noise", type=str, default="True", help="whether to add noise")
parser.add_argument("--kernel", type=str, default="rbf", help="kernel")
parser.add_argument("--gpu", type=int, default=5, help="gpu")
parser.add_argument("--seed", type=int, default=0, help="random seed")
parser.add_argument("--suffix", type=str, default="", help="suffix for res folder")
parser.add_argument("--method", type=str, default="svgd", help="svgd, gsvgd or s-svgd")
parser.add_argument("--save_every", type=int, default=100, help="batch size")

args = parser.parse_args([])

In [3]:
dim = args.dim
lr = args.lr
lr_g = args.lr_g
delta = args.delta
T = args.T
nparticles = args.nparticles
epochs = args.epochs
seed = args.seed
eff_dims = 10
add_noise = True if args.noise == "True" else False
save_every = args.save_every  # save metric values every 100 epochs
print(f"Running for lr: {lr}, nparticles: {nparticles}")

device = torch.device(f"cuda:{args.gpu}" if args.gpu != -1 else "cpu")


Running for lr: 0.0001, nparticles: 2500


In [4]:
results_folder = f"./res/diffusion{args.suffix}/{args.kernel}_epoch{epochs}_lr{lr}_delta{delta}_n{nparticles}_dim{dim}"
results_folder = f"{results_folder}/seed{seed}"

In [5]:
results_folder

'./res/diffusion/rbf_epoch20000_lr0.0001_delta0.1_n2500_dim50/seed0'

In [6]:
torch.__version__

'1.13.0'

In [7]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

if args.kernel == "rbf":
    Kernel = RBF
    BatchKernel = BatchRBF

In [8]:
print(f"Device: {device}")
torch.manual_seed(seed)
np.random.seed(seed)

## initialize conditional diffusion
beta = 5
sigma = 0.1

distribution = Diffusion(dim, beta, device=device)

loc = torch.arange(0, dim+1, 1)[1:]
distribution.loc = loc
noise_covariance = torch.diag(sigma**2 * torch.ones(len(loc), device=device))
distribution.set_noise_covariance(noise_covariance)
distribution.set_C()

x_true = distribution.brownian_motion((1, dim))
u_true = distribution.solution(x_true)

obs_noise = torch.normal(0, sigma, size=(1, len(loc))).to(device)
obs = u_true[:, loc] + obs_noise
distribution.obs = obs

# initialize particles
x0 = distribution.brownian_motion((nparticles, dim))

C = distribution.C.cpu().numpy()
dt = distribution.dt.cpu().numpy()
loc = distribution.loc.cpu().numpy()
beta = distribution.beta




Device: cuda:5


In [9]:
## SVGD
print("Running SVGD")
x = x0.clone().requires_grad_()
# sample from variational density
kernel = Kernel(method="med_heuristic")

Running SVGD


In [10]:
## SVGD
print("Running SVGD")
x = x0.clone().requires_grad_()
# sample from variational density
kernel = Kernel(method="med_heuristic")
svgd = SVGD(distribution, kernel, optim.Adam([x], lr=lr), device=device)

start = time.time()
svgd.fit(x, epochs=20000, save_every=save_every)
elapsed_time = time.time() - start

fitted_method = svgd
#particles = fitted_method.particles
particles = x

method_name = "svgd"


print(x)
## save results
pickle.dump(
     {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
           **{"time": elapsed_time}
    },
    open(results_folder + f"/particles_svgd1.p", "wb")
)

# target distribution
torch.save(distribution, results_folder + '/target_dist.p')
print(x.shape)

Running SVGD


100%|██████████| 20000/20000 [10:16<00:00, 32.46it/s]

tensor([[-0.2692, -0.3656, -0.4887,  ...,  0.4843,  0.3027,  0.2843],
        [-0.1733, -0.2772, -0.5696,  ..., -0.0802, -0.1565, -0.1156],
        [-0.1535, -0.3225, -0.4577,  ..., -0.0598, -0.1530,  0.0283],
        ...,
        [-0.2528, -0.2558, -0.6194,  ..., -0.0967, -0.2763, -0.0594],
        [-0.1253, -0.3272, -0.5237,  ..., -0.0474, -0.2390, -0.1437],
        [-0.2528, -0.3644, -0.5053,  ...,  0.0945, -0.0265,  0.0605]],
       device='cuda:5', requires_grad=True)
torch.Size([2500, 50])


In [11]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 2).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_2_test1.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:30:51<00:00,  3.67it/s]

tensor([[-0.0715, -0.1665, -0.4459,  ...,  0.0613, -0.0440,  0.0245],
        [-0.2074, -0.2750, -0.5714,  ..., -0.1338, -0.1893, -0.1060],
        [-0.1984, -0.3573, -0.4334,  ..., -0.0656, -0.2037, -0.0663],
        ...,
        [-0.2001, -0.3777, -0.6189,  ..., -0.0667, -0.3347, -0.1990],
        [-0.1986, -0.4318, -0.5799,  ..., -0.1362, -0.3726, -0.2132],
        [-0.1589, -0.2619, -0.4966,  ...,  0.0714, -0.1458, -0.1304]],
       device='cuda:5')


In [12]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 5).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_5_test1.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:32:15<00:00,  3.61it/s]

tensor([[-0.2239, -0.3058, -0.4494,  ...,  0.0658, -0.0865,  0.0212],
        [-0.3468, -0.4639, -0.6301,  ..., -0.1213, -0.1832, -0.1060],
        [-0.1477, -0.2598, -0.4251,  ..., -0.0830, -0.2405, -0.0663],
        ...,
        [-0.1417, -0.2833, -0.5261,  ..., -0.1271, -0.3310, -0.1990],
        [-0.1880, -0.3806, -0.6039,  ..., -0.1581, -0.3664, -0.2132],
        [-0.2433, -0.3320, -0.5114,  ...,  0.1051, -0.0921, -0.1304]],
       device='cuda:5')


In [13]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 10).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_10_test1.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:33:49<00:00,  3.55it/s]

tensor([[-0.2405, -0.3927, -0.5661,  ...,  0.0681, -0.0817,  0.0279],
        [-0.1167, -0.2426, -0.4575,  ..., -0.1846, -0.1791, -0.1027],
        [-0.1866, -0.3012, -0.4838,  ..., -0.0488, -0.2297, -0.0562],
        ...,
        [-0.2994, -0.4231, -0.5866,  ..., -0.1554, -0.2457, -0.1956],
        [-0.1047, -0.2818, -0.4968,  ..., -0.0868, -0.4074, -0.2065],
        [-0.2745, -0.4250, -0.5849,  ..., -0.0035, -0.0756, -0.1237]],
       device='cuda:5')


In [14]:
## S-SVGD
# sample from variational density
print("Running S-SVGD")
x_s_svgd = x0.clone().requires_grad_()
s_svgd = SlicedSVGD(distribution, device=device)

start = time.time()
x_s_svgd, metric_s_svgd = s_svgd.fit(
    samples=x_s_svgd, 
    n_epoch=epochs, 
    lr=lr_g,
    eps=lr,
    save_every=save_every
)
elapsed_time = time.time() - start

fitted_method = s_svgd
particles = x_s_svgd

pickle.dump(
{
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_ssvgd_test1.p", "wb")
)

print(particles)

Running S-SVGD


100%|██████████| 20000/20000 [2:15:57<00:00,  2.45it/s]  

tensor([[-0.2034, -0.4376, -0.6029,  ...,  0.0293, -0.1166, -0.0124],
        [-0.0013, -0.0760, -0.5483,  ..., -0.2364, -0.2322, -0.1408],
        [-0.1099, -0.2020, -0.5740,  ..., -0.0930, -0.2628, -0.0967],
        ...,
        [-0.4006, -0.4963, -0.5805,  ..., -0.1962, -0.2836, -0.2343],
        [-0.0633, -0.3916, -0.4267,  ..., -0.1254, -0.4446, -0.2458],
        [-0.2864, -0.5481, -0.6218,  ..., -0.0427, -0.1120, -0.1542]],
       device='cuda:5', grad_fn=<CloneBackward0>)
